# 📊 **Pipeline de Processamento de Linguagem Natural (NLP)**

## **Do Texto Bruto à Modelagem Preditiva**

Neste projeto aplico técnicas clássicas de Processamento de Linguagem Natural (NLP)
para transformar textos em representações numéricas e aplicar modelos de Machine Learning.

O objetivo é demonstrar domínio prático do pipeline completo e análise de texto com técnicas de NLP.

Cada etapa está organizada com descrição conceitual e explicação do que está sendo realizado.



---

## 🔎 **Pipeline do Projeto**

Este notebook segue as seguintes etapas:

1. **Preparação dos dados** – `pandas`, `numpy` e carregamento da base.  

2. **Pré-processamento textual** – `regex`, `nltk` (stopwords), padronização para lowercase e limpeza de caracteres.  

3. **Vetorização** – `CountVectorizer` (Bag of Words) e `TfidfVectorizer` (TF-IDF) para transformação do texto em matriz esparsa.  

4. **Similaridade** – Similaridade do cosseno (`cosine similarity`) para medir proximidade entre documentos.  

5. **Modelagem** – `LogisticRegression` (scikit-learn) para classificação supervisionada.  

6. **Avaliação** – `classification_report` e `confusion_matrix` para análise de desempenho.

---

# **Importação das Bibliotecas** 🧰

In [ ]:
# 📊 Manipulação de dados
import pandas as pd
import numpy as np

# 🧹 Processamento de texto
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# 🔢 Vetorização (Representação Numérica)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 🧠 Modelos Baseados em Transformers
from transformers import pipeline

# 📐 Similaridade entre documentos
from sklearn.metrics.pairwise import cosine_similarity

# 🧠 Topic Modeling
from sklearn.decomposition import LatentDirichletAllocation

# 🤖 Modelagem e Avaliação (Machine Learning Clássico)
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

## 🔹 **Etapa 1 – Coleta e Carregamento dos Dados** 💾

Nesta seção realizamos a clonagem do repositório público contendo o dataset
de reviews de e-commerce.

Os dados utilizados são **avaliações de produtos do Mercado Livre**,
armazenadas em formato JSON.

O código abaixo:

- Clona o repositório via GitHub
- Lista os arquivos disponíveis
- Carrega os arquivos JSON
- Concatena os dados em um único DataFrame para análise

In [ ]:
!git clone https://github.com/octaprice/ecommerce-product-dataset.git
!ls ecommerce-product-dataset/data/mercadolivre_com_br


df1 = pd.read_json("ecommerce-product-dataset/data/mercadolivre_com_br/reviews_mercadolivre_com_br_1.json")
df2 = pd.read_json("ecommerce-product-dataset/data/mercadolivre_com_br/reviews_mercadolivre_com_br_2.json")
df = pd.concat([df1, df2], ignore_index=True)

## 🔹 **Etapa 2 – Modelos de Emoção e Análise de Sentimento** 😊🎭

Nesta etapa utilizamos modelos baseados em **Transformers** capaz de capturar melhor o **contexto das frases** para analisar textos
sob duas perspectivas:

- 🎭 **Análise de Emoção** → identifica emoções específicas **(alegria, tristeza, raiva, medo, surpresa, nojo)**.
- 😊 **Análise de Sentimento** → classifica a polaridade geral **(positivo, neutro, negativo)**.

🤖 **Carregamento do Modelo de Emoção** 🎭

In [ ]:
classificador_emocao = pipeline(
    task="text-classification",
    model="bhadresh-savani/distilbert-base-uncased-emotion",
)

🔎 **Função para Detecção de Emoção**

Recebe um texto e retorna a emoção predominante identificada pelo modelo pré-treinado.

In [ ]:
def detectar_emocao(texto):

    texto = str(texto)[:512]
    resultado = classificador_emocao(texto)[0]

    return resultado['label']

📊 **Aplicação do Modelo em uma Amostra do Dataset**

In [ ]:
# Seleciona amostra para demonstração
df_demo = df.sample(200, random_state=42).copy()

# Executa inferência em lote
resultados = classificador_emocao(
    df_demo["content"].astype(str).tolist()
)

# Armazena emoções previstas
df_demo["emocao"] = [r["label"] for r in resultados]

df_demo.head()

🤖 **Carregamento do Modelo de Sentimento** 😊

In [ ]:
classificador_sentimento = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment"
)

def detectar_sentimento(texto):
    texto = str(texto)[:512]
    resultado = classificador_sentimento(texto)[0]
    return resultado['label']

📊 **Aplicação do Modelo na Mesma Amostra**

In [ ]:
resultados_sentimento = classificador_sentimento(
    df_demo["content"].astype(str).tolist()
)

df_demo["sentimento"] = [r["label"] for r in resultados_sentimento]

**👀 Visualização e Análise das Emoções**

In [ ]:
df_demo[['content','emocao','sentimento']].sample(5)

In [ ]:
df_demo['sentimento'].value_counts().reset_index()

# 🔹 **Etapa 3 – Detecção de Inconsistências** ⚠️



### 📊 **Objetivo**
Identificar contradições entre a **nota (rating)** e o **sentimento previsto no texto**.

Consideramos inconsistência quando:

- ⭐ Nota alta (4 ou 5) → sentimento negativo  
- ⭐ Nota baixa (1 ou 2) → sentimento positivo  

Essa análise ajuda a identificar possíveis erros de rotulagem,
ironia textual ou limitações do modelo.

🔎 **Definição das regras de inconsistência**

In [ ]:
inconsistencias = df_demo[
    (
        (df_demo["rating"] >= 4) & (df_demo["sentimento"] == "negative")
    )
    |
    (
        (df_demo["rating"] <= 2) & (df_demo["sentimento"] == "positive")
    )
]

# Visualizar exemplos
inconsistencias[["content", "rating", "sentimento"]].head()

In [ ]:
print('Inconsistentes:', len(inconsistencias))
print('%:', round(len(inconsistencias)/len(df_demo)*100, 2))

### 📌 **Interpretação**

A presença de inconsistências pode indicar:

- Uso irônico da linguagem
- Ambiguidade textual
- Diferença entre percepção emocional e avaliação final
- Limitações do modelo pré-treinado

Essa etapa adiciona uma camada analítica ao projeto,
indo além da simples classificação automática.

**Reviews muito curtas apresentaram maior proporção de inconsistências, sugerindo que a baixa quantidade de contexto textual pode dificultar a interpretação correta pelo modelo.**

# 🔹 **Etapa 4 – Detecção de Temas (Topic Modeling)** 🧩

### 📌 **Objetivo**
Identificar automaticamente os principais temas presentes nas reviews,
utilizando modelagem não supervisionada.

Aplicamos:
- **Bag of Words (CountVectorizer)** para transformar texto em matriz numérica  
- **LDA (Latent Dirichlet Allocation)** para descobrir grupos de palavras recorrentes  

Essa abordagem permite responder:

> “Quais são os principais assuntos discutidos nas avaliações?”

### 🔢 **Vetorização (Bag of Words)**

In [ ]:
# Stopwords em português
nltk.download("stopwords")
stopwords_pt = stopwords.words("portuguese")

# Vetorização do texto
vetorizador = CountVectorizer(
    stop_words=stopwords_pt,
    max_features=1000
)

matriz_texto = vetorizador.fit_transform(df_demo["content"])

### **🧠 Treinamento do Modelo LDA**

In [ ]:
# Treinamento do modelo de tópicos
modelo_lda = LatentDirichletAllocation(
    n_components=5,
    random_state=42
)

modelo_lda.fit(matriz_texto)

### 🔎 **Principais Palavras por Tema**

In [ ]:
# Visualizar palavras mais relevantes de cada tema
palavras = vetorizador.get_feature_names_out()

for indice_tema, tema in enumerate(modelo_lda.components_):
    principais_palavras = [palavras[i] for i in tema.argsort()[-10:]]
    print(f"Tema {indice_tema+1}")
    print(principais_palavras)

### 📊 **Análise dos Temas (Amostra)**

Identificamos o **tema dominante** de cada review com base na maior
probabilidade atribuída pelo LDA.

Em seguida, analisamos a distribuição dos temas
e sua relação com o sentimento.

In [ ]:
# 4. Identificar tema principal de cada review
matriz_temas = modelo_lda.transform(matriz_texto)

df_demo["tema_principal"] = matriz_temas.argmax(axis=1)

**Análise Estatística Descritiva dos temas**

In [ ]:
df_demo["tema_principal"].value_counts().reset_index()

In [ ]:
df_demo.groupby('tema_principal')['rating'].mean().reset_index()

### 😊 **Distribuição de Sentimento por Tema**

Analisamos como o sentimento se distribui dentro de cada tema,
permitindo identificar quais assuntos estão mais associados
a avaliações positivas ou negativas.

In [ ]:
df_demo.groupby(["tema_principal", "sentimento"]).size().unstack().reset_index()

## 🔹 **Etapa 5 – Análise de Similaridade Textual** 📐

**🎯 Objetivo**

Nesta etapa utilizamos **TF-IDF** para representar os textos
e aplicamos **similaridade do cosseno** para medir proximidade entre reviews.

Essa abordagem permite identificar textos semanticamente semelhantes,
servindo como base para sistemas de recomendação ou busca textual.

🔢 **Representação vetorial com TF-IDF**


In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_pt = stopwords.words("portuguese")


tfidf_vectorizer = TfidfVectorizer(
    stop_words=stopwords_pt,
    min_df=5
)

X_tfidf = tfidf_vectorizer.fit_transform(df["content"])

📚 **Vocabulário criado pelo modelo**

In [ ]:
vocab_tfidf = tfidf_vectorizer.get_feature_names_out()

📐 **Similaridade do cosseno entre duas reviews**


In [ ]:
i, j = 11, 20

print("Review A:\n", df.loc[i, "content"])
print("\nReview B:\n", df.loc[j, "content"])

similaridade = cosine_similarity(
    X_tfidf[i],
    X_tfidf[j]
)[0][0]

print("\nSimilaridade:", similaridade)

🔎 **Encontrar textos semelhantes a uma review específica**

In [ ]:
idx_base = 11

# Calcular similaridade com todas as reviews
similaridades = cosine_similarity(
    X_tfidf[idx_base],
    X_tfidf
)[0]

# Ordenar por maior similaridade
df_sim = (
    df.assign(similaridade=similaridades)
      .sort_values("similaridade", ascending=False)
)

# Top 5 mais semelhantes (excluindo a própria)
df_sim.iloc[1:6][["rating", "similaridade", "content"]]

🔍 **Busca semântica a partir de um texto digitado**

In [ ]:
consulta = "produto deixou meu cabelo macio e cheiroso"

vetor_consulta = tfidf_vectorizer.transform([consulta])

similaridades_consulta = cosine_similarity(
    vetor_consulta,
    X_tfidf
)[0]

df_busca = (
    df.assign(similaridade=similaridades_consulta)
      .sort_values("similaridade", ascending=False)
)

df_busca.head(5)[["rating", "similaridade", "content"]]

# 🔹 **Etapa 6 – Classificação de Texto com Regressão Logística** 🤖

Nesta etapa treinamos um modelo supervisionado para classificar reviews
em **negativo, neutro ou positivo**, a partir da nota atribuída.

**Essa abordagem é amplamente utilizada em:**
- Classificação de reviews
- Detecção de spam
- Categorização de tickets
- Identificação de intenção em chatbots

### 🎯 **Criação da Variável Alvo**

Para transformar o problema em uma tarefa de classificação supervisionada,
convertimos a nota numérica (`rating`) em categorias de sentimento:

- 1 ou 2 → **negativo**
- 3 → **neutro**
- 4 ou 5 → **positivo**

**Aqui definimos a classe que o modelo irá prever, baseada na nota atribuída pelo usuário.**

In [ ]:
def classificar_rating(r):
    if r <= 2:
        return "negativo"
    elif r == 3:
        return "neutro"
    else:
        return "positivo"

df["classe_sentimento"] = df["rating"].apply(classificar_rating)

### 🔎 **Definição das variáveis**

Aqui separamos o que será usado como entrada **(texto)**
e o que o modelo deve prever **(classe de sentimento)**.

In [ ]:
X = df["content"].astype(str)
y = df["classe_sentimento"]

### ✂️ **Divisão em Treino e Teste**

Separamos os dados em **80% treino** e **20% teste**.

O objetivo é treinar o modelo em uma parte dos dados
e avaliar seu desempenho em dados não vistos garantindo uma avaliação mais confiável.


In [ ]:
from sklearn.model_selection import train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

### 🔤 **Vetorização com TF-IDF**

- Transformação do texto em representação numérica.

- max_features=5000 limita o vocabulário

- O modelo passa a trabalhar com vetores numéricos

In [ ]:
vetorizador = TfidfVectorizer(
    max_features=5000,
    stop_words=None
)

X_treino_vet = vetorizador.fit_transform(X_treino)
X_teste_vet = vetorizador.transform(X_teste)

### 📈 **Treinamento do Modelo**

Treinamos uma Regressão Logística com **balanceamento de classes** para lidar com desbalanceamento do dataset.

In [ ]:
modelo = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    random_state=42
)

modelo.fit(X_treino_vet, y_treino)

### 🔮**Realização das Previsões**
Aplicamos o modelo no conjunto de teste.

**A variável predicoes** foi utilizada na etapa de avaliação do modelo, sendo essencial para o **cálculo das métricas de desempenho**, como **accuracy** e **relatório de classificação**, podendo ainda ser explorada em análises adicionais de erro.

In [ ]:
predicoes = modelo.predict(X_teste_vet)

### 📊**Avaliação do Modelo**

Avaliamos o desempenho utilizando:

- Accuracy
- Precision
- Recall
- F1-score

In [ ]:
print("Accuracy:", accuracy_score(y_teste, predicoes))
print("\nRelatório de classificação:")
print(classification_report(y_teste, predicoes))

### 📊 **Interpretação dos Resultados**

O modelo alcançou **86% de acurácia**, com desempenho elevado na classe **positiva**, que concentra a maior parte das amostras.

As classes **negativa** e principalmente **neutra** apresentaram métricas inferiores, refletindo o desbalanceamento do dataset.

Foi utilizado `class_weight="balanced"` na Regressão Logística para tentar reduzir esse efeito, ajustando o peso das classes minoritárias durante o treinamento. Ainda assim, observa-se que o desbalanceamento impacta o desempenho final.

### 🧪**Teste com Novos Textos**

Aplicação prática do modelo em exemplos manuais.

In [ ]:
# Exemplo 1
texto = ['Gostei muito, indico.']
texto_vet = vetorizador.transform(texto)
print("Predição:", modelo.predict(texto_vet)[0])

# Exemplo 2
texto = ['Até que é bom.']
texto_vet = vetorizador.transform(texto)
print("Predição:", modelo.predict(texto_vet)[0])

# Exemplo 3
texto = ['Péssimo produto, odiei o cheiro.']
texto_vet = vetorizador.transform(texto)
print("Predição:", modelo.predict(texto_vet)[0])

# **📌 Conclusão**

Neste projeto aplicamos um pipeline completo de NLP, incluindo:

- Pré-processamento textual  
- Representação vetorial (BoW e TF-IDF)  
- Modelagem supervisionada  
- Topic Modeling (LDA)  
- Similaridade textual  
- Análise com Transformers  

O teste com novos textos demonstra a aplicação prática do modelo,
mostrando sua capacidade de generalizar para avaliações não vistas.

Como próximos passos, seria possível:
- Ajustar hiperparâmetros  
- Aplicar técnicas de balanceamento mais avançadas  
- Comparar com modelos baseados em Transformers  

**O projeto evidencia a aplicação integrada de técnicas clássicas e modernas de NLP em um cenário real de reviews de e-commerce.**